In [14]:
import os
from PublicDataReader import Fred
import pandas as pd

In [15]:

# FRED API 키
api_key = os.getenv("FRED_API_KEY")

# 인스턴스 생성하기
api = Fred(api_key)

In [3]:
search_text = "consumer price index"
result = api.get_data(api_name="series_search", search_text=search_text)
result.head()

,id,realtime_start,realtime_end,title,observation_start,observation_end,frequency,frequency_short,units,units_short,seasonal_adjustment,seasonal_adjustment_short,last_updated,popularity,group_popularity,notes
0,CPIAUCSL,2025-07-02,2025-07-02,Consumer Price Index for All Urban Consumers: ...,1947-01-01,2025-05-01,Monthly,M,Index 1982-1984=100,Index 1982-1984=100,Seasonally Adjusted,SA,2025-06-11 07:41:40-05,95,95,The Consumer Price Index for All Urban Consume...
1,CPIAUCNS,2025-07-02,2025-07-02,Consumer Price Index for All Urban Consumers: ...,1913-01-01,2025-05-01,Monthly,M,Index 1982-1984=100,Index 1982-1984=100,Not Seasonally Adjusted,NSA,2025-06-11 07:41:40-05,73,95,Handbook of Methods (https://www.bls.gov/opub/...
2,CUUS0000SA0,2025-07-02,2025-07-02,Consumer Price Index for All Urban Consumers: ...,1984-01-01,2024-07-01,Semiannual,SA,Index 1982-1984=100,Index 1982-1984=100,Not Seasonally Adjusted,NSA,2025-01-15 07:40:23-06,46,95,NaN
3,FPCPITOTLZGUSA,2025-07-02,2025-07-02,"Inflation, consumer prices for the United States",1960-01-01,2024-01-01,Annual,A,Percent,%,Not Seasonally Adjusted,NSA,2025-04-16 13:53:02-05,85,85,Inflation as measured by the consumer price in...
4,CORESTICKM159SFRBATL,2025-07-02,2025-07-02,Sticky Price Consumer Price Index less Food an...,1968-01-01,2025-05-01,Monthly,M,Percent Change from Year Ago,% Chg. from Yr. Ago,Seasonally Adjusted,SA,2025-06-11 12:01:03-05,84,84,The Sticky Price Consumer Price Index (CPI) is...


In [4]:
# 시리즈 ID 값
series_id = "CPIAUCNS"

# 시리즈 데이터 조회
df = api.get_data(api_name="series_observations", series_id=series_id)
df.tail()

,realtime_start,realtime_end,date,value
1344,2025-06-12,2025-06-12,2025-01-01,317.671
1345,2025-06-12,2025-06-12,2025-02-01,319.082
1346,2025-06-12,2025-06-12,2025-03-01,319.799
1347,2025-06-12,2025-06-12,2025-04-01,320.795
1348,2025-06-12,2025-06-12,2025-05-01,321.465


In [5]:
"""
미국 소비자 물가 지수 CPI(Consumer Price Index)
시리즈명: Consumer Price Index for All Urban Consumers: All Items in U.S. City Average
시리즈명(번역): 모든 도시 소비자를 위한 소비자 물가 지수: 미국 도시 평균의 모든 품목
단위: 지수 1982-1984=100, 계절에 따라 조정되지 않음.
빈도: 월간
FRED URL: https://fred.stlouisfed.org/series/CPIAUCNS
출처: 미국 노동 통계국(U.S. Bureau of Labor Statistics)

"""


def CPI() -> pd.DataFrame:
    """CPI(Consumer Price Index) 시리즈 데이터 조회 함수"""
    
    # 시리즈 ID 값
    series_id = "CPIAUCNS"
    
    # 시리즈 데이터 조회
    df = api.get_data(
        api_name="series_observations", 
        series_id=series_id
    )
    
    # value 컬럼 숫자형 변환
    df['value'] = pd.to_numeric(df['value'], errors="coerce")
    # date 컬럼 날짜형 변환
    df['date'] = pd.to_datetime(df['date'])
    # date 컬럼을 인덱스로 설정
    df = df.set_index("date")

    # 전년 동월 값 컬럼 생성
    df['value_last_year'] = df['value'].shift(12)
    df['CPI(YoY)'] = (df['value'] - df['value_last_year']) / df['value_last_year'] * 100
    
    # 전년 동월 값 컬럼만 선택
    df = df[['CPI(YoY)']]
    return df


In [6]:
"""
미국 개인 소비 지출 PCE(Personal Consumption Expenditures)
시리즈명: Personal Consumption Expenditures: Chain-type Price Index
시리즈명(번역): 개인 소비 지출: 체인형 물가지수
단위: 지수 2012=100, 계절에 따라 조정됨.
빈도: 월간
FRED URL: https://fred.stlouisfed.org/series/PCEPI
출처: 미국 경제 분석국 (U.S. Bureau of Economic Analysis)

"""

def PCE() -> pd.DataFrame:
    """PCE(Personal Consumption Expenditures) 시리즈 데이터 조회 함수"""
    
    # 시리즈 ID 값
    series_id = "PCEPI"
    
    # 시리즈 데이터 조회
    df = api.get_data(
        api_name="series_observations", 
        series_id=series_id
    )
    
    # value 컬럼 숫자형 변환
    df['value'] = pd.to_numeric(df['value'], errors="coerce")
    # date 컬럼 날짜형 변환
    df['date'] = pd.to_datetime(df['date'])
    # date 컬럼을 인덱스로 설정
    df = df.set_index("date")

    # 전년 동월 값 컬럼 생성
    df['value_last_year'] = df['value'].shift(12)
    df['PCE(YoY)'] = (df['value'] - df['value_last_year']) / df['value_last_year'] * 100
    
    # 전년 동월 값 컬럼만 선택
    df = df[['PCE(YoY)']]
    return df

In [7]:
"""
미국 생산자 물가 지수 PPI(Producer Price Index)
시리즈명: Producer Price Index by Commodity: Final Demand
시리즈명(번역): 상품별 생산자 물가 지수: 최종 수요
단위: 지수 2009년 11월=100, 계절에 따라 조정되지 않음.
빈도: 월간
FRED URL: https://fred.stlouisfed.org/series/PPIFID
출처: 미국 노동 통계국(Source: U.S. Bureau of Labor Statistics)

"""

def PPI() -> pd.DataFrame:
    """PPI(Producer Price Index) 시리즈 데이터 조회 함수"""
    
    # 시리즈 ID 값
    series_id = "PPIFID"
    
    # 시리즈 데이터 조회
    df = api.get_data(
        api_name="series_observations", 
        series_id=series_id
    )
    
    # value 컬럼 숫자형 변환
    df['value'] = pd.to_numeric(df['value'], errors="coerce")
    # date 컬럼 날짜형 변환
    df['date'] = pd.to_datetime(df['date'])
    # date 컬럼을 인덱스로 설정
    df = df.set_index("date")

    # 전년 동월 값 컬럼 생성
    df['value_last_year'] = df['value'].shift(12)
    df['PPI(YoY)'] = (df['value'] - df['value_last_year']) / df['value_last_year'] * 100
    
    # 전년 동월 값 컬럼만 선택
    df = df[['PPI(YoY)']]
    return df

In [8]:
"""
미국 연준 기준금리
시리즈명: Federal Funds Target Range - Upper Limit
시리즈명(번역): 연방 기금 목표 범위 - 상한
단위: 백분율, 계절에 따라 조정되지 않음.
빈도: 매일, 7일.
FRED URL: https://fred.stlouisfed.org/series/DFEDTARU
출처: 미국 연방준비제도 이사회(Board of Governors of the Federal Reserve System (US)
"""

def FED_RATE() -> pd.DataFrame:
    """목표 연준 기준금리 시리즈 데이터 조회 함수"""
    
    # 시리즈 ID 값
    series_id = "DFEDTARU"
    
    # 시리즈 데이터 조회
    df = api.get_data(
        api_name="series_observations", 
        series_id=series_id
    )
    
    # value 컬럼 숫자형 변환 컬럼 생성
    df['FED RATE'] = pd.to_numeric(df['value'], errors="coerce")
    # date 컬럼 날짜형 변환
    df['date'] = pd.to_datetime(df['date'])
    
    # date 컬럼을 인덱스로 설정
    df = df.set_index('date')
    # 기준금리 컬럼만 선택
    df = df[['FED RATE']]
    return df

In [9]:
"""
케이스-쉴러 미국 주택 가격 지수 Case-Shiller U.S. Home Price Index
시리즈명: S&P/Case-Shiller U.S. National Home Price Index
시리즈명(번역): S&P/케이스-실러 미국 전국 주택 가격 지수
단위: 지수 1월 2000=100, 계절에 따라 조정됨.
빈도: 월간
FRED URL: https://fred.stlouisfed.org/series/CSUSHPISA
출처: S&P 다우존스 인덱스 LLC(S&P Dow Jones Indices LLC)

"""

def CS() -> pd.DataFrame:
    """케이스-쉴러 전국 주택 가격 지수(미국) 시리즈 데이터 조회 함수"""
    
    # 시리즈 ID 값
    series_id = "CSUSHPISA"
    
    # 시리즈 데이터 조회
    df = api.get_data(
        api_name="series_observations", 
        series_id=series_id
    )
    
    # value 컬럼 숫자형 변환
    df['value'] = pd.to_numeric(df['value'], errors="coerce")
    # date 컬럼 날짜형 변환
    df['date'] = pd.to_datetime(df['date'])
    # date 컬럼을 인덱스로 설정
    df = df.set_index("date")

    # 전년 동월 값 컬럼 생성
    df['value_last_year'] = df['value'].shift(12)
    df['CS(YoY)'] = (df['value'] - df['value_last_year']) / df['value_last_year'] * 100
    
    # 전년 동월 값 컬럼만 선택
    df = df[['CS(YoY)']]
    return df

In [10]:
"""
미국 경제 성장률(직전 분기 대비 GDP 성장률(연율))
시리즈명: Real Gross Domestic Product
시리즈명(번역): 실질 국내 총생산
단위: 이전 기간 대비 변화율, 계절 조정 연간 비율
빈도: 분기별
FRED URL: https://fred.stlouisfed.org/series/A191RL1Q225SBEA
출처: 미국 경제 분석국(Source: U.S. Bureau of Economic Analysis)
"""

def GDP() -> pd.DataFrame:
    """미국 GDP 성장률(전분기 대비 연율) 시리즈 데이터 조회 함수"""
    
    # 시리즈 ID 값
    series_id = "A191RL1Q225SBEA"
    
    # 시리즈 데이터 조회
    df = api.get_data(
        api_name="series_observations", 
        series_id=series_id
    )
    
    # value 컬럼 숫자형 변환
    df['GDP RATE'] = pd.to_numeric(df['value'], errors="coerce")
    # date 컬럼 날짜형 변환
    df['date'] = pd.to_datetime(df['date'])
    # date 컬럼을 인덱스로 설정
    df = df.set_index("date")
    # GDP 성장률 컬럼만 선택
    df = df[['GDP RATE']]
    return df

In [11]:
cpi = CPI()
pce = PCE()
ppi = PPI()
fed_rate = FED_RATE()
cs = CS()
gdp = GDP()

In [12]:
# 데이터 병합
dfs = [cpi, pce, ppi, fed_rate, cs, gdp]
# 날짜 기준을 일별로 변환 후 결측값은 이전 값으로 대체
dfs = list(map(lambda x: x.resample("D").asfreq().ffill(), dfs))
# 데이터 병합
df = pd.concat(dfs, axis=1).ffill()
# 기간 설정
start_date = "2018-01-01"
target = df[start_date:]

target.head()

,CPI(YoY),PCE(YoY),PPI(YoY),FED RATE,CS(YoY),GDP RATE
date,,,,,,
2018-01-01,2.070508,1.757649,2.598566,1.5,6.162997,3.3
2018-01-02,2.070508,1.757649,2.598566,1.5,6.162997,3.3
2018-01-03,2.070508,1.757649,2.598566,1.5,6.162997,3.3
2018-01-04,2.070508,1.757649,2.598566,1.5,6.162997,3.3
2018-01-05,2.070508,1.757649,2.598566,1.5,6.162997,3.3


In [13]:
import plotly.express as px
fig = px.line(target, x=target.index, y=list(target.columns))

fig.update_layout(
    title={
        'text': "글로벌 주요 경제 지표",
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title="날짜",
    yaxis_title="값",
)

fig.show()